In [1]:
import pandas as pd
import numpy as np
import gc
import os
import open3d as o3d
import matplotlib.pyplot as plt

In [2]:
%matplotlib notebook

In [3]:
DYNAMIC_NPY_PATH = "/home/saby/Projects/ati/data/data/datasets/IROSpaper/self-seg/crop_img.npy"
RECONST_NPY_PATH = "/home/saby/Projects/ati/data/data/datasets/IROSpaper/self-seg/reconSt.npy"

In [4]:
def from_polar_np(velo):
    angles = np.linspace(0, np.pi * 2, velo.shape[-1])
    dist, z = velo[:, 0], velo[:, 1]
    x = np.cos(angles) * dist
    y = np.sin(angles) * dist
    out = np.stack([x,y,z], axis=1)
    return out.astype('float32')

def draw_pcd(pcd, where='opn_nb'):
    if where is 'opn_nb':
        visualizer = o3d.JVisualizer()
        visualizer.add_geometry(pcd)
        visualizer.show()
    elif where is 'opn_view':
        o3d.visualization.draw_geometries([pcd], width=1280, height=800)
    elif where is 'mat_3d':
        plt.figure()
        pts = np.asarray(pcd.points)
        plt.scatter(pts[:,0], pts[:,1], pts[:,2])
        plt.grid()
        plt.show()
    elif where is 'mat_2d':
        plt.figure()
        pts = np.asarray(pcd.points)
        plt.scatter(pts[:,0], pts[:,1])
        plt.grid()
        plt.show()

In [5]:
dynamic_arr = np.load(DYNAMIC_NPY_PATH)
dynamic_arr.shape

(64, 2, 40, 512)

In [6]:
reconst_arr = np.load(RECONST_NPY_PATH)
reconst_arr.shape

(64, 2, 40, 512)

In [7]:
idx = 60
thresh = 0.045

In [8]:
dynamic_polar_img = dynamic_arr[idx]
reconst_polar_img = reconst_arr[idx]

In [9]:
dynamic_polar_img.shape

(2, 40, 512)

In [10]:
reconst_polar_img.shape

(2, 40, 512)

In [11]:
mask_img = (np.abs(dynamic_polar_img[0] - reconst_polar_img[0]) > thresh).astype(int)
color_arr = np.concatenate((np.expand_dims(mask_img, axis=0), np.zeros((1,40,512)), np.zeros((1,40,512))), axis=0)
color_arr.shape

(3, 40, 512)

In [12]:
dynamic_img = from_polar_np(np.expand_dims(dynamic_polar_img, axis=0))[0]
reconst_img = from_polar_np(np.expand_dims(reconst_polar_img, axis=0))[0]

dynamic_img.shape, reconst_img.shape

((3, 40, 512), (3, 40, 512))

In [13]:
new_pcd = o3d.geometry.PointCloud()
new_pcd.points = o3d.utility.Vector3dVector(dynamic_img.reshape((3,-1)).T*120)
new_pcd.colors = o3d.utility.Vector3dVector(color_arr.reshape((3,-1)).T)

In [14]:
draw_pcd(new_pcd)

JVisualizer with 1 geometries